## 📚 Prerequisites

Before running this notebook, ensure you have configured Azure AI services, set the appropriate configuration parameters, and set up a Conda environment to ensure reproducibility. You can find the setup instructions and how to create a Conda environment in the [REQUIREMENTS.md](REQUIREMENTS.md) file.


In [1]:
import os

# Define the target directory (change yours)
target_directory = r"C:\Users\zshirah\repos\gbbai-chat-with-your-database"

# Check if the directory exists
if os.path.exists(target_directory):
    # Change the current working directory
    os.chdir(target_directory)
    print(f"Directory changed to {os.getcwd()}")
else:
    print(f"Directory {target_directory} does not exist.")

Directory changed to C:\Users\zshirah\repos\gbbai-chat-with-your-database


In [2]:
from src.database.azuresql import AzureSQLManager

DATABASE = "dev-sql-server"
az_sql_client = AzureSQLManager(DATABASE)

In [3]:
from src.aoai.azure_openai import AzureOpenAIManager

az_client = AzureOpenAIManager()

## Knowledge Engineering

In this section, we will discuss the process of knowledge engineering in the context of our project. This involves extracting tables from databases, converting them into natural language, and vectorizing the content. The vectorization process enhances the relevance of our search system by enabling semantic search capabilities. This way, we can create rules and knowledge bases for our artificial intelligence systems to provide more accurate and contextually relevant search results.

In [4]:
tables_names = az_sql_client.get_table_names()
schema = az_sql_client.process_schema(tables_names[0])

In [6]:
schema


{'detroit_tigers_baseball_stats': [{'column_number': 0,
   'column_name': 'name',
   'data_type': 'nvarchar'},
  {'column_number': 1, 'column_name': 'position', 'data_type': 'nvarchar'},
  {'column_number': 2, 'column_name': 'Games_Played', 'data_type': 'tinyint'},
  {'column_number': 3, 'column_name': 'At_Bats', 'data_type': 'smallint'},
  {'column_number': 4, 'column_name': 'Hits', 'data_type': 'tinyint'},
  {'column_number': 5, 'column_name': 'Home_Runs', 'data_type': 'tinyint'},
  {'column_number': 6, 'column_name': 'RBIs', 'data_type': 'tinyint'},
  {'column_number': 7,
   'column_name': 'Batting_Average',
   'data_type': 'float'}]}

In [5]:
system_message = """You are an AI assistant that helps people find information.  Act as a Senior Database Engineer, capable of understanding complex 
relationships between fields and tables. Your task involves interpreting and analyzing 
the schema of SQL database tables provided in JSON format. You examine the structure,
understand the purpose of each field, and identify the interconnections between fields. 
Your objective is to produce a comprehensive summary that elucidates the table's design
and its potential applications. This summary should highlight the table's intended use, 
emphasizing the key data it contains. Your expertise is essential for demystifying the 
navigation and optimization of complex database systems, facilitating users to comprehend
the significance and operational context of their data assets efficiently. Take your time."""

In [7]:
query = f"""
Please examine the schema of the table thoroughly. Provide an in-depth analysis that not only explains the design and structure of the table but also outlines its potential applications and the types of queries it can support.
Also provide atleast 10 queries in natural lengauge that can be run on the table.
Please take your time.

Here is the schema of the table: {schema}
"""

In [8]:
content = az_client.generate_chat_response(
    conversation_history=[],
    query=query,
    system_message_content=system_message,
    max_tokens=1000,
)

2024-02-07 14:02:41,449 - micro - MainProcess - INFO     Sending request to OpenAI with query: 
Please examine the schema of the table thoroughly. Provide an in-depth analysis that not only explains the design and structure of the table but also outlines its potential applications and the types of queries it can support.
Also provide atleast 10 queries in natural lengauge that can be run on the table.
Please take your time.

Here is the schema of the table: {'detroit_tigers_baseball_stats': [{'column_number': 0, 'column_name': 'name', 'data_type': 'nvarchar'}, {'column_number': 1, 'column_name': 'position', 'data_type': 'nvarchar'}, {'column_number': 2, 'column_name': 'Games_Played', 'data_type': 'tinyint'}, {'column_number': 3, 'column_name': 'At_Bats', 'data_type': 'smallint'}, {'column_number': 4, 'column_name': 'Hits', 'data_type': 'tinyint'}, {'column_number': 5, 'column_name': 'Home_Runs', 'data_type': 'tinyint'}, {'column_number': 6, 'column_name': 'RBIs', 'data_type': 'tinyint'

2024-02-07 14:03:16,102 - micro - MainProcess - INFO     Received response from OpenAI: The provided schema is for a database table named 'detroit_tigers_baseball_stats'. This table seems to be designed to store statistics for players of the Detroit Tigers Baseball team. The table has eight columns, each with a distinct purpose.

1. 'name' - This is a NVARCHAR type column, used to store the names of the players. It allows for storing alphanumeric characters and can handle names with special characters.

2. 'position' - This is also a NVARCHAR type column, used to store the position that each player plays in the game. It can include positions like pitcher, catcher, infielder, outfielder, etc.

3. 'Games_Played' - This is a TINYINT type column. It is used to store the number of games each player has played. The TINYINT data type is used for efficiency as it uses less storage and the number of games a player can play in a season is relatively small.

4. 'At_Bats' - This is a SMALLINT type

In [10]:
embedding = az_client.generate_embedding(input_text=content)

In [11]:
embedding

[-0.01131843589246273,
 0.019958358258008957,
 0.006321978289633989,
 -0.019958358258008957,
 -0.041098009794950485,
 0.03027406893670559,
 -0.02840597741305828,
 -0.022581927478313446,
 0.0025686256121844053,
 -0.04694953188300133,
 0.01796664297580719,
 -0.0016122589586302638,
 -0.010453070513904095,
 0.005810313392430544,
 0.01958748698234558,
 -0.002999591641128063,
 0.0051269493997097015,
 -0.0029635345563292503,
 0.021029764786362648,
 -0.01917540840804577,
 0.002980704652145505,
 0.014491444453597069,
 -0.0123074259608984,
 0.024312660098075867,
 -0.01741720549762249,
 0.005429140757769346,
 0.017060069367289543,
 -0.01236236933618784,
 -0.008784150704741478,
 0.0042615835554897785,
 0.017788076773285866,
 -0.007149570155888796,
 -0.009498420171439648,
 -0.03060373291373253,
 -0.01785675622522831,
 0.008035539649426937,
 -0.001677504857070744,
 0.022059962153434753,
 0.0304938443005085,
 0.03291137516498566,
 0.023900581523776054,
 0.0005249714595265687,
 -0.013042300008237362,


In [12]:
len(embedding)

1536

## Indexing

In [13]:
import os
import random
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

True

In [14]:
# Set the service endpoint and API key from the environment
# Create an SDK client
endpoint = os.environ["AZURE_AI_SEARCH_SERVICE_ENDPOINT"]
search_client = SearchClient(
    endpoint=endpoint,
    index_name="query-dev-index",
    credential=AzureKeyCredential(os.environ["AZURE_SEARCH_ADMIN_KEY"]),
)

In [15]:
tables_names = az_sql_client.get_table_names()
schema = az_sql_client.process_schema(tables_names[0])

In [16]:
chunked_content_docs = []

In [17]:
import random

# Iterate over each table name
for table in tables_names:
    # Process the schema of the table
    schema = az_sql_client.process_schema(table)

    # Generate a chat response
    content = az_client.generate_chat_response(
        conversation_history=[],
        query=query,
        system_message_content=system_message,
        max_tokens=1000,
    )
    # Generate a random document ID
    doc_id = str(random.randint(1, 1000000))

    # Generate an embedding for the content
    embedding = az_client.generate_embedding(input_text=content)

    # Create a JSON object with the table content, vector, document ID, and table name
    json_data = {
        "table_content": content,
        "table_vector": embedding,
        "document_id": doc_id,
        "table_name": table,
    }

    # Append the JSON object to the list of documents
    chunked_content_docs.append(json_data)

2024-02-07 14:07:03,948 - micro - MainProcess - INFO     Sending request to OpenAI with query: 
Please examine the schema of the table thoroughly. Provide an in-depth analysis that not only explains the design and structure of the table but also outlines its potential applications and the types of queries it can support.
Also provide atleast 10 queries in natural lengauge that can be run on the table.
Please take your time.

Here is the schema of the table: {'detroit_tigers_baseball_stats': [{'column_number': 0, 'column_name': 'name', 'data_type': 'nvarchar'}, {'column_number': 1, 'column_name': 'position', 'data_type': 'nvarchar'}, {'column_number': 2, 'column_name': 'Games_Played', 'data_type': 'tinyint'}, {'column_number': 3, 'column_name': 'At_Bats', 'data_type': 'smallint'}, {'column_number': 4, 'column_name': 'Hits', 'data_type': 'tinyint'}, {'column_number': 5, 'column_name': 'Home_Runs', 'data_type': 'tinyint'}, {'column_number': 6, 'column_name': 'RBIs', 'data_type': 'tinyint'

2024-02-07 14:07:44,702 - micro - MainProcess - INFO     Received response from OpenAI: The provided schema is for a table named 'detroit_tigers_baseball_stats', which likely stores statistical data related to the players of the Detroit Tigers, a professional baseball team. The table contains eight columns, each representing a different aspect of a player's performance or role within the team.

1. 'name': This field is of data type 'nvarchar', which means it stores non-Unicode character data. It's likely used to store the names of the players.

2. 'position': This 'nvarchar' field probably represents the role or position a player holds within the team (such as pitcher, catcher, infielder, etc).

3. 'Games_Played': This 'tinyint' field stores the number of games a player has participated in. The 'tinyint' data type means this field can hold values from 0 to 255.

4. 'At_Bats': This 'smallint' field likely records the number of attempts a player has had to hit the ball. The 'smallint' da

In [18]:
print(f"Uploading batch of {len(chunked_content_docs)} documents...")
result = search_client.upload_documents(documents=chunked_content_docs)
for res in result:
    print("Upload of new document succeeded: {}".format(res.succeeded))

Uploading batch of 2 documents...
Upload of new document succeeded: True
Upload of new document succeeded: True
